In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Configuration graphique
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
print("Bibliothèques importées avec succès")

Bibliothèques importées avec succès


In [13]:
# Charger le jeu de données California Housing
housing = fetch_california_housing(as_frame=True)
df = housing.frame
print(f"Dimensions du dataset : {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"\nVariable cible : MedHouseVal (valeur médiane du logement en centaines de milliers de $)")
print(f"\nDescription des variables :")
print("-" * 60)
descriptions = {
'MedInc': 'Revenu médian du quartier',
'HouseAge': 'Âge médian des logements',
'AveRooms': 'Nombre moyen de pièces par logement',
'AveBedrms': 'Nombre moyen de chambres par logement',
'Population': 'Population du quartier',
'AveOccup': 'Nombre moyen d\'occupants par logement',
'Latitude': 'Latitude géographique',
'Longitude': 'Longitude géographique',
'MedHouseVal': 'Prix médian du logement (cible)'
}
for col, desc in descriptions.items():
    print(f"  {col:15s} → {desc}")

Dimensions du dataset : 20640 lignes × 9 colonnes

Variable cible : MedHouseVal (valeur médiane du logement en centaines de milliers de $)

Description des variables :
------------------------------------------------------------
  MedInc          → Revenu médian du quartier
  HouseAge        → Âge médian des logements
  AveRooms        → Nombre moyen de pièces par logement
  AveBedrms       → Nombre moyen de chambres par logement
  Population      → Population du quartier
  AveOccup        → Nombre moyen d'occupants par logement
  Latitude        → Latitude géographique
  Longitude       → Longitude géographique
  MedHouseVal     → Prix médian du logement (cible)


In [15]:
# Afficher les premières lignes avec .head()
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [16]:
# Utiliser .info() pour vérifier les types de colonnes
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [17]:
# Utiliser .describe() pour obtenir les statistiques descriptives :
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [19]:
# 2.Nettoyage
# Vérifier les valeurs manquantes avec .isnull().sum()
df.isnull().sum()
# Si présentes : soit les supprimer (.dropna), soit les imputer (.fillna)
df.dropna()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [ ]:
# Vérifier et supprimer les doublons avec .duplicated() et .drop_duplicates()

In [ ]:
# Détecter les valeurs aberrantes (outliers) à l'aide de boxplots
# Les boxplots montrent visuellement la distribution et les points extrêmes

In [ ]:
# Filtrer les outliers extrêmes pour AveRooms, AveBedrms et AveOccup
# On fixe des seuils raisonnables (ex: AveRooms < 50, AveOccup < 20)
# pour éviter que ces valeurs extrêmes faussent le modèle

In [ ]:
# 3. Analyse exploratoire (EDA)
# Tracer la distribution de la variable cible (MedHouseVal)
# avec un histogramme + courbe de densité (kde)
# et un boxplot pour repérer la médiane et les extrêmes

In [1]:
# Calculer et afficher la matrice de corrélation sous forme de heatmap
# Cela permet d'identifier quelles variables sont le plus liées au prix :
#   - corrélation proche de +1 → relation positive forte
#   
#   - corrélation proche de -1 → relation négative forte- corrélation proche de  0 → pas de relation linéaire

In [2]:
# Scatter plots : croiser les variables les plus corrélées avec la cible
# Ex : MedInc vs MedHouseVal → on s'attend à une relation positive forte
# (plus le revenu est élevé, plus le logement est cher)

In [3]:
# Carte géographique : afficher chaque quartier (Latitude, Longitude)
# coloré selon le prix médian, pour visualiser la répartition spatiale
# des prix en Californie (zones côtières = plus cher)

In [4]:
# 4. Ingénierie des variables
# Créer de nouvelles variables dérivées, par exemple :
#   PiecesParChambre = AveRooms / AveBedrms  (ratio pièces/chambres)
#   PopParLogement   = Population / AveOccup  (estimation du nb de logements)
# Ces nouvelles features peuvent capturer des informations supplémentaires

In [5]:
# Séparer les données en :
#   X = variables explicatives (features) → les 10 colonnes d'entrée
#   y = variable cible → MedHouseVal (le prix à prédire)

In [6]:
# 5. Séparation entraînement / test
# Diviser X et y en deux sous-ensembles avec train_test_split :
#   - 80% pour l'entraînement (le modèle apprend sur ces données)
#   - 20% pour le test (on évalue la qualité des prédictions)
# Le paramètre random_state=42 garantit la reproductibilité

In [ ]:
# 6. Construction du modèle de régression linéaire
# Instancier LinearRegression() et entraîner avec .fit(X_train, y_train)
# Le modèle calcule les coefficients β optimaux en minimisant
# la somme des carrés des erreurs (méthode des moindres carrés)

# Afficher :
#   - model.intercept_ → l'ordonnée à l'origine (β₀)
#   - model.coef_ → les coefficients (β₁, β₂, ..., βₙ)
# Un coefficient positif = la variable augmente le prix
# Un coefficient négatif = la variable diminue le prix

In [8]:
# 7. Prédiction et évaluation 
# Prédire les prix sur l'ensemble de test avec model.predict(X_test)
# Puis calculer les métriques d'évaluation :
#
#   MSE  = moyenne des (réel - prédit)²     → erreur quadratique moyenne
#   
#   RMSE = √MSE     -> plus interprétable (même unité que y)       
# MAE  = moyenne des |réel - prédit|    ->  erreur absolue moyenne    
# R²   = 1 - (SS_res / SS_tot)          -> proportion de variance expliquée
#
# R² = 1.0 → prédiction parfaite
# R² = 0.0 → le modèle ne fait pas mieux que la moyenne

In [9]:
# 8. Visualisation des résultats
# Graphique 1 : Valeurs réelles vs Prédites
# Un scatter plot avec la ligne diagonale (prédiction parfaite)
# Plus les points sont proches de la diagonale, meilleur est le modèle

In [10]:
# Graphique 2 : Analyse des résidus (résidu = réel - prédit)
#   a) Histogramme des résidus → devrait suivre une distribution normale centrée sur 0
#   
#      b) Résidus vs Prédictions → les points doivent être dispersés aléatoirement (un pattern visible indiquerait que le modèle rate quelque chose)

In [11]:
# Graphique 3 : Barplot des coefficients du modèle
# Permet de visualiser l'importance relative de chaque variable :
#   Coefficient positif = contribue à augmenter le prix
#   Coefficient négatif = contribue à diminuer le prix

In [ ]:
# 9. Export des résultats
# Créer un DataFrame de résultats contenant :
#   - les features du test
#   - le prix réel
#   - le prix prédit
#   - le résidu (écart)
#   - l'erreur en pourcentage
# Puis exporter en CSV avec .to_csv("resultats_regression.csv")